# सेमांटिक कर्नेल में एजेंट स्क्रैचपैड के साथ चैट इतिहास को कम करना

यह नोटबुक दिखाती है कि सेमांटिक कर्नेल की चैट इतिहास को कम करने की सुविधा का उपयोग कैसे करें, साथ ही एजेंट स्क्रैचपैड का उपयोग करके बातचीत के दौरान संदर्भ बनाए रखें। यह लंबे संवादों को संभालने वाले कुशल AI एजेंट बनाने के लिए आवश्यक है, जो टोकन सीमा को पार न करें।

## आप क्या सीखेंगे:
1. **चैट इतिहास को कम करना**: टोकन उपयोग को प्रबंधित करने के लिए बातचीत के इतिहास को स्वचालित रूप से सारांशित करना
2. **एजेंट स्क्रैचपैड**: उपयोगकर्ता की प्राथमिकताओं और पूर्ण किए गए कार्यों को ट्रैक करने के लिए एक स्थायी मेमोरी सिस्टम
3. **टोकन उपयोग ट्रैकिंग**: यह देखना कि इतिहास को कम करने के साथ और बिना टोकन उपयोग कैसे बदलता है

## आवश्यकताएँ:
- पर्यावरण वेरिएबल्स के साथ Azure OpenAI सेटअप
- पिछले पाठों से एजेंट की बुनियादी अवधारणाओं की समझ


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## एजेंट स्क्रैचपैड को समझना

### एजेंट स्क्रैचपैड क्या है?

**एजेंट स्क्रैचपैड** एक स्थायी मेमोरी सिस्टम है जिसे एजेंट उपयोग करते हैं:
- **पूरे किए गए कार्यों को ट्रैक करना**: उपयोगकर्ता के लिए किए गए कार्यों को रिकॉर्ड करना
- **उपयोगकर्ता की प्राथमिकताओं को संग्रहीत करना**: पसंद, नापसंद और आवश्यकताओं को याद रखना
- **संदर्भ बनाए रखना**: बातचीत के दौरान महत्वपूर्ण जानकारी को सुलभ रखना
- **दोहराव को कम करना**: बार-बार वही सवाल पूछने से बचना

### यह कैसे काम करता है:
1. **लिखने की प्रक्रिया**: एजेंट नई जानकारी सीखने के बाद स्क्रैचपैड को अपडेट करता है
2. **पढ़ने की प्रक्रिया**: निर्णय लेते समय एजेंट स्क्रैचपैड की सलाह लेता है
3. **स्थायित्व**: जानकारी तब भी बनी रहती है जब चैट इतिहास कम हो जाता है

इसे एजेंट की व्यक्तिगत नोटबुक के रूप में सोचें, जो बातचीत के इतिहास को पूरा करती है।


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## एजेंट स्क्रैचपैड प्लगइन बनाएं

यह प्लगइन एजेंट को एक स्थायी स्क्रैचपैड फ़ाइल पढ़ने और लिखने की अनुमति देता है।


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## चैट इतिहास को कम करना प्रारंभ करें

ChatHistorySummarizationReducer स्वचालित रूप से बातचीत के इतिहास को संक्षेप में प्रस्तुत करता है जब यह एक निर्धारित सीमा से अधिक हो जाता है।


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## छुट्टी योजना एजेंट बनाएं

यह एजेंट उपयोगकर्ताओं को छुट्टियों की योजना बनाने में मदद करेगा और स्क्रैचपैड के माध्यम से संदर्भ बनाए रखेगा।


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## छुट्टी योजना वार्तालाप चलाएं

अब आइए एक पूरी वार्तालाप का प्रदर्शन करें जिसमें शामिल हैं:
1. प्रारंभिक योजना अनुरोध
2. प्राथमिकताओं का संग्रह
3. यात्रा कार्यक्रम बनाना
4. स्थान परिवर्तन
5. चैट इतिहास को कम करना
6. स्क्रैचपैड का उपयोग


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## परिणामों का विश्लेषण

आइए देखें कि हमारी बातचीत के दौरान क्या हुआ:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## मुख्य बातें

### 1. चैट इतिहास में कमी
- **स्वचालित सक्रियता**: संदेशों की संख्या सीमा से अधिक होने पर कमी होती है
- **टोकन की बचत**: सारांश के बाद टोकन उपयोग में महत्वपूर्ण कमी
- **संदर्भ संरक्षण**: सारांश में महत्वपूर्ण जानकारी संरक्षित रहती है

### 2. एजेंट स्क्रैचपैड के लाभ
- **स्थायी मेमोरी**: उपयोगकर्ता की प्राथमिकताएँ इतिहास में कमी के बाद भी बनी रहती हैं
- **कार्य ट्रैकिंग**: एजेंट पूरा किए गए कार्यों का रिकॉर्ड रखता है
- **बेहतर अनुभव**: प्राथमिकताओं को बार-बार दोहराने की आवश्यकता नहीं

### 3. टोकन उपयोग के पैटर्न
- **रेखीय वृद्धि**: प्रत्येक संदेश के साथ टोकन की संख्या बढ़ती है
- **नाटकीय गिरावट**: कमी के बाद टोकन की संख्या में उल्लेखनीय गिरावट
- **सतत बातचीत**: सीमाओं के भीतर लंबी बातचीत को सक्षम बनाता है


## सफाई

इस डेमो के दौरान बनाए गए स्क्रैचपैड फाइल को साफ करें:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# सारांश

बधाई हो! आपने सफलतापूर्वक एक AI एजेंट को उन्नत संदर्भ प्रबंधन क्षमताओं के साथ लागू किया है:

## आपने क्या सीखा:
- **चैट इतिहास को संक्षिप्त करना**: टोकन सीमाओं को प्रबंधित करने के लिए बातचीत को स्वचालित रूप से संक्षेपित करना  
- **एजेंट स्क्रैचपैड**: उपयोगकर्ता की प्राथमिकताओं और पूर्ण कार्यों के लिए स्थायी मेमोरी लागू करना  
- **टोकन प्रबंधन**: लंबी बातचीत में टोकन उपयोग को ट्रैक और अनुकूलित करना  
- **संदर्भ संरक्षण**: बातचीत को संक्षिप्त करते समय महत्वपूर्ण जानकारी को बनाए रखना  

## वास्तविक दुनिया में उपयोग:
- **ग्राहक सेवा बॉट्स**: सत्रों के बीच ग्राहक की प्राथमिकताओं को याद रखना  
- **निजी सहायक**: चल रहे प्रोजेक्ट्स और उपयोगकर्ता की आदतों को ट्रैक करना  
- **शैक्षिक ट्यूटर**: छात्र की प्रगति और सीखने की प्राथमिकताओं को बनाए रखना  
- **स्वास्थ्य सेवा सहायक**: टोकन सीमाओं का सम्मान करते हुए रोगी का इतिहास बनाए रखना  

## अगले कदम:
- अधिक उन्नत स्क्रैचपैड स्कीमाओं को लागू करें  
- बहु-उपयोगकर्ता परिदृश्यों के लिए डेटाबेस स्टोरेज जोड़ें  
- डोमेन-विशिष्ट आवश्यकताओं के लिए कस्टम संक्षेपण रणनीतियाँ बनाएं  
- सेमांटिक मेमोरी सर्च के लिए वेक्टर डेटाबेस के साथ संयोजन करें  
- ऐसे एजेंट बनाएं जो दिनों बाद भी पूरी संदर्भ जानकारी के साथ बातचीत फिर से शुरू कर सकें  



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।  
